In [43]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-18.0.2'
os.environ['SPARK_HOME'] = 'C:\Program Files\spark-3.3.0-bin-hadoop3'

In [44]:
#Create a spark session

spark = SparkSession.builder.appName('Read JSON File into DataFrame').getOrCreate()

#Read JSON and assign to pyspark dataframes

customer_df = spark.read.json('cdw_sapp_custmer.json')
credit_df = spark.read.json('cdw_sapp_credit.json')
branch_df = spark.read.json('cdw_sapp_branch.json')

In [45]:

customer_df.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [48]:
from pyspark.sql.functions import concat_ws
#Format customer data
customer_df = customer_df.withColumnRenamed("FIRST_NAME","First_Name")
customer_df = customer_df.withColumnRenamed("MIDDLE_NAME", "Middle_Name")
customer_df = customer_df.withColumnRenamed("LAST_NAME", "Last_Name")

customer_df = customer_df.withColumn('FULL_STREET_ADDRESS', concat_ws(' ', customer_df.STREET_NAME, customer_df.APT_NO))
customer_df = customer_df.drop('APT_NO', 'STREET_NAME')


customer_df = customer_df.withColumn("CUST_ZIP",customer_df.CUST_ZIP.cast("int"))
customer_df.show(2)
customer_df.printSchema()

+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|First_Name|Last_Name|        LAST_UPDATED|Middle_Name|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|    6109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|   Redwood Drive 829|
+----------------+------------+-------------+-------------------+----------+----------+---